In [13]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [14]:
def read_data(path_to_csv):
    df = pd.read_csv(path_to_csv, header=0, index_col=0)
    
    # select columns by type
    float_cols = df.select_dtypes(include=['float64']).columns
    str_cols = df.select_dtypes(include=['object']).columns
    
    # fill NAs
    df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)
    
    # convert str to bool for true/false values
    mask = df.applymap(type) != bool
    bool_to_str = {True: 1, False: 0}
    df = df.where(mask, df.replace(bool_to_str))
    
    return df

In [15]:
def get_unique_values(df):
    sequence_list = [list(row) for row in df['sequence']]
    unique_items = set([item for sublist in sequence_list for item in sublist])

    return unique_items

In [16]:
def get_sequence_info(df, unique_items):
    list_dicts = []
    input_target_values = []
    list_already_seen = []

    for row in range(0, len(df)):
    
        for position, char in enumerate(df.loc[row, 'sequence']):
            new_row_nr = row + position
            char_dict = {}
            sequence = df.loc[row, 'sequence']
        
            # specific for position in sequence
            start_coords_col = 'start_coords' + str(position+1)
            char_dict['start_coords.x'] = df.loc[row, str(start_coords_col + '.x')]
            char_dict['start_coords.y'] = df.loc[row, str(start_coords_col + '.y')]
            char_dict['start_coords.z'] = df.loc[row, str(start_coords_col + '.z')]
            
            char_dict['row'] = row
        
            if position == 0:
                input_value = '<start>'
                target_value = char
                input_target_values.append([input_value, target_value])
                
                list_already_seen.append([])
            
            else:
                input_value = df.loc[row, 'sequence'][position-1]
                target_value = char
                input_target_values.append([input_value, target_value])

                list_already_seen.append(df.loc[row, 'sequence'][:position])
                
            list_dicts.append(char_dict)
    
    return list_dicts, input_target_values, list_already_seen

In [17]:
def get_row_info(df):
    dicts_row = {}
    
    for row in range(0, len(df)):
        row_dict = {}
        sequence = df.loc[row, 'sequence']
        
        for position, char in enumerate(df.loc[row, 'sequence']):
            char_coordinates_x = 'coordinates_' + char + '.x'
            char_coordinates_y = 'coordinates_' + char + '.y'
            char_coordinates_z = 'coordinates_' + char + '.z'
            row_dict[char_coordinates_x] = df.loc[row, char_coordinates_x]
            row_dict[char_coordinates_y] = df.loc[row, char_coordinates_y]
            row_dict[char_coordinates_z] = df.loc[row, char_coordinates_z]
            
            char_containment = char + '.containment'
            char_strong_k = char + '.strong_k'
            char_mid_k = char + '.mid_k'
            char_food_k = char + '.food_k'
            row_dict[char_containment] = df.loc[row, char_containment]
            row_dict[char_food_k] = df.loc[row, char_food_k]
            row_dict[char_strong_k] = df.loc[row, char_strong_k]
            row_dict[char_mid_k] = df.loc[row, char_mid_k]
        
            dicts_row[row] = row_dict
    
    return dicts_row        

In [20]:
def create_singlestep_df(list_dicts, input_target_values, list_already_seen):
    single_step_df = pd.DataFrame(list_dicts)
    single_step_df.insert(loc=0, column='input', value=0)
    single_step_df.insert(loc=1, column='target', value=0)

    for row in range(0, len(input_target_values)):
        single_step_df.loc[row, 'input'] = input_target_values[row][0]
        single_step_df.loc[row, 'target'] = input_target_values[row][1]
        
    for row, elem in enumerate(list_already_seen):
        for item in unique_items:
            single_step_df.loc[row, str(item + '.already_seen')] = 0
            if item in elem:
                single_step_df.loc[row, str(item + '.already_seen')] = 1
                
    for row in range(0, len(single_step_df)):
        for key, values in dicts_row.items():
            if single_step_df.loc[row, 'row'] == key:
                for k,v in values.items():
                    single_step_df.loc[row, k] = v
                    
    single_step_df = single_step_df.drop('row', axis=1)
                    
    return single_step_df

In [21]:
df = read_data('data/task_environments_2021-07-15_transformed.csv')
unique_items = get_unique_values(df)

list_dicts, input_target_values, list_already_seen = get_sequence_info(df, unique_items)
dicts_row = get_row_info(df)

single_step_df = create_singlestep_df(list_dicts, input_target_values, list_already_seen)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [22]:
single_step_df

,input,target,start_coords.x,start_coords.y,start_coords.z,l.already_seen,d.already_seen,g.already_seen,f.already_seen,r.already_seen,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,h,h,0.000000,4.000000,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,h,s,0.000000,4.000000,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,s,d,0.000000,2.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,d,g,0.000000,1.000000,3.000000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
date = datetime.today().strftime('%Y-%m-%d')
filename = 'data/single_step_df_ints_' + str(date) + '.csv'

filename

'data/single_step_df_ints_2022-03-15.csv'

In [24]:
single_step_df.to_csv(filename, index=False, header=True)